<a href="https://colab.research.google.com/github/sakhri99/data-scraping-/blob/main/GBV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install unidecode 

     |████████████████████████████████| 245kB 4.0MB/s 


In [4]:
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from unidecode import unidecode
import time
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np

In [5]:
consumer_key = 'YOUR-CONSUMER-KEY'
consumer_secret = 'YOUR-CONSUMER-SECRET'
access_token = 'YOUR-ACCESS-TOKEN'
access_secret = 'YOUR-ACCESS-SECRET'

In [6]:
auth = tweepy.OAuthHandler('YOUR-CONSUMER-KEY', 'YOUR-CONSUMER-SECRET')
auth.set_access_token('YOUR-CONSUMER-SECRET','YOUR-ACCESS-SECRET')
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
def tweetSearch(query, limit):
    tweets  =[]
    for page in tweepy.Cursor(api.search, q=query, count=limit, tweet_mode="extended" ).pages(limit):
      for tweet in page:
        tweets.append(tweet)
    return tweets

In [8]:
def tweets_to_data_frame(tweets):
  df = pd.DataFrame(data=[tweet.full_text.encode('utf-8') for tweet in tweets], columns=["Tweets"])
  df["id"] = np.array([tweet.id for tweet in tweets])
  df["lens"] = np.array([len(tweet.full_text) for tweet in tweets])
  df["date"] = np.array([tweet.created_at for tweet in tweets])
  df["place"] = np.array([tweet.place for tweet in tweets])
  df["coordinateS"] = np.array([tweet.coordinates for tweet in tweets])
  df["lang"] = np.array([tweet.lang for tweet in tweets])
  df["source"] = np.array([tweet.source for tweet in tweets])
  df["likes"] = np.array([tweet.favorite_count for tweet in tweets])
  df["retweets"] = np.array([tweet.retweet_count for tweet in tweets])
  return df

In [9]:
mkdir data

mkdir: cannot create directory ‘data’: File exists


In [10]:
hashtags = ['#GBV', '#sexism', '#rape' , '#endgbv', '#genderbasedviolence' , '#YessAllWomen' , '#MeeToo' ,'#CGSHEquity', '#GBVFridays', '#YOUTHAGAINSTGBV','#GenerationEquality' ,'#combattingviolentextremism','#sexualabuse','#domesticvilence','#sexualviolence','#harass','#sexualharrassement','#womenphysicalviolence','#sexualassult','#sextrafficking','#forcedmarriage','#allgbv']


In [11]:
from tqdm import tqdm

In [12]:
total_tweets = 0

for n in tqdm(hashtags):
  hash_tweets = tweetSearch(query=n,limit=7000)
  total_tweets += int(len(hash_tweets))
  # second we convert our tweets into datarame
  df = tweets_to_data_frame(hash_tweets)
  #third we save the dataframe into csv file
  df.to_csv("data/{}_tweets.csv".format(n))
  # show total number of tweets collected 
print("total_tweets: {}".format(total_tweets))

100%|██████████| 22/22 [15:52<00:00, 43.28s/it]

total_tweets: 20376


In [13]:
assault_df = pd.read_csv('/content/data/#rape_tweets.csv')


In [14]:
assault_df.head()


,Unnamed: 0,Tweets,id,lens,date,place,coordinateS,lang,source,likes,retweets
0,0,b'RT @WomensPeaceNW: \xf0\x9f\x9b\x91The decad...,1393172888400891907,140,2021-05-14 11:54:35,NaN,NaN,en,Twitter for Android,0,28
1,1,"b""\xe0\xa4\x85\xe0\xa4\xb8\xe0\xa5\x8d\xe0\xa4...",1393172092867145729,283,2021-05-14 11:51:25,NaN,NaN,hi,TweetDeck,0,0
2,2,b'RT @Calinka005: #Morocco\xe2\x80\x99s viciou...,1393171297383993352,140,2021-05-14 11:48:15,NaN,NaN,en,Twitter Web App,0,5
3,3,b'RT @BabySingh99: \xe0\xa4\xaf\xe0\xa5\x87 \x...,1393170925290344461,138,2021-05-14 11:46:47,NaN,NaN,hi,Twitter for iPhone,0,11
4,4,b'RT @thatsTamil: \xe0\xae\x95\xe0\xaf\x81\xe0...,1393170595542491138,140,2021-05-14 11:45:28,NaN,NaN,ta,Twitter Web App,0,4


In [15]:
assault_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6218 entries, 0 to 6217
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   6218 non-null   int64  
 1   Tweets       6218 non-null   object 
 2   id           6218 non-null   int64  
 3   lens         6218 non-null   int64  
 4   date         6218 non-null   object 
 5   place        20 non-null     object 
 6   coordinateS  0 non-null      float64
 7   lang         6218 non-null   object 
 8   source       6218 non-null   object 
 9   likes        6218 non-null   int64  
 10  retweets     6218 non-null   int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 534.5+ KB


In [19]:
pip install cufflinks

In [21]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [23]:
import re


In [16]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [24]:
assault_df['clean_tweet'] = assault_df['Tweets'].apply(lambda x: clean_tweet(x))

In [27]:
assault_df['clean_tweet'].head()


0    b RT xf0 x9f x9b x91The decades long impunity ...
1    b xe0 xa4 x85 xe0 xa4 xb8 xe0 xa5 x8d xe0 xa4 ...
2    b RT Morocco xe2 x80 x99s vicious MoroccanPoli...
3    b RT xe0 xa4 xaf xe0 xa5 x87 xe0 xa4 xb9 xe0 x...
4    b RT xe0 xae x95 xe0 xaf x81 xe0 xae xb3 xe0 x...
Name: clean_tweet, dtype: object

In [28]:
n=3000
print('Original tweet:\n'+ assault_df['Tweets'][n])
print()
print('Clean tweet:\n'+assault_df['clean_tweet'][n])
print()

Original tweet:
b'RT @majidahhashim: Hi @PDRMsia, this is a threat to #rape a child. This predator must be stopped. Rape is not a joke. All rape threats must\xe2\x80\xa6'

Clean tweet:
b RT Hi this is a threat to rape a child This predator must be stopped Rape is not a joke All rape threats must xe2 x80 xa6



In [29]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

In [32]:
assault_df['Sentiment'] = assault_df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

In [34]:
assault_df['Sentiment'].value_counts().iplot(kind='bar', xTitle='Sentiment',
                                    yTitle='Count', title='Overall Sentiment Distribution')